In [1]:
import pandas as pd
import numpy as np

import matplotlib.cm as cm
import matplotlib.colors as colors


#get wikiped tables

In [2]:
#get tables from wikipedia
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.sample(n=5, random_state=1)

,Postal Code,Borough,Neighbourhood
116,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
112,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
99,M1P,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town..."
161,M9X,Not assigned,Not assigned
35,M9E,Not assigned,Not assigned


In [3]:
#drop not assigned borough
df = df[df.Borough != 'Not assigned']
df.sample(n=5, random_state=1)

,Postal Code,Borough,Neighbourhood
94,M5N,Central Toronto,Roselawn
65,M3K,North York,Downsview
64,M2K,North York,Bayview Village
104,M6P,West Toronto,"High Park, The Junction South"
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [4]:
#replace not assigned neighbourhood with borough
df[df.Neighbourhood == 'Not assigned']
#welp that's easy lol, none of them are

,Postal Code,Borough,Neighbourhood


##all borough with NA values is within all neighbourhood with NA values

In [5]:
df.shape

(103, 3)

#get lat and long from csv

In [6]:
geo = pd.read_csv('drive/MyDrive/coursera/Geospatial_Coordinates.csv')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
#join column and reset index
df = df.set_index('Postal Code').join(geo.set_index('Postal Code')).reset_index()
df.head(5)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [8]:
import folium
from geopy.geocoders import Nominatim

In [9]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.65238435, -79.38356765.


In [10]:
map_torronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_torronto)  
    
map_torronto

#simple clustering based on borough

count # of unique borough and use them as cluster count

In [11]:
k = len(pd.unique(df.Borough))
k

10

In [12]:
df_dummies = pd.get_dummies(df.Borough)

In [13]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=k, random_state=42).fit(df_dummies)

# check cluster labels generated for each row in the dataframe
df['labels'] = kmeans.labels_
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,labels
0,M3A,North York,Parkwoods,43.753259,-79.329656,1
1,M4A,North York,Victoria Village,43.725882,-79.315572,1
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0
...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,2
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,8
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,2


In [14]:
map_torronto = folium.Map(location=[latitude, longitude], zoom_start=10)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
for lat, lng, borough, neighborhood, labels in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood'], df['labels']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[labels-1],
        fill=True,
        fill_color=rainbow[labels-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_torronto)  
    
map_torronto

##there it is a cluster based on borough :D